In [1]:
# -*- coding: utf-8 -*-

import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import pickle

In [2]:
data=pd.read_excel("movie data_new.xlsx")
data

,Unnamed: 0,Movie Name,Year of Release,Watch Time,Genre,Movie Rating,Metascore of movie,Director,Cast,Votes,Description
0,0,Free Guy,2021,115 min,"\nAction, Comedy, Sci-Fi",7.6,62.0,Shawn Levy,"Ryan Reynolds,Jodie Comer,Taika Waititi","29,441",A bank teller discovers that he's actually an ...
1,1,The Suicide Squad,2021,132 min,"\nAction, Adventure, Comedy",7.4,72.0,James Gunn,"Margot Robbie,Idris Elba,John Cena","156,225","Supervillains Harley Quinn, Bloodsport, Peacem..."
2,2,Reminiscence,2021,116 min,"\nMystery, Romance, Sci-Fi",5.9,46.0,Lisa Joy,"Hugh Jackman,Rebecca Ferguson,Thandiwe Newton","11,879","Nick Bannister, a private investigator of the ..."
3,3,Beckett,2021,110 min,"\nAction, Crime, Drama",5.6,52.0,Ferdinando Cito Filomarino,"John David Washington,Boyd Holbrook,Vicky Krieps","14,267","Following a tragic car accident in Greece, Bec..."
4,4,Eternals,2021,NaN,"\nAction, Adventure, Drama",NaN,NaN,Chloé Zhao,"Richard Madden,Salma Hayek,Angelina Jolie",NaN,"The saga of the Eternals, a race of immortal b..."
...,...,...,...,...,...,...,...,...,...,...,...
995,995,The Good Shepherd,2006,167 min,"\nDrama, History, Thriller",6.7,61.0,Robert De Niro,"Matt Damon,Angelina Jolie,Robert De Niro","100,403",The tumultuous early history of the Central In...
996,996,Falling,I 2020,112 min,\nDrama,6.4,63.0,Viggo Mortensen,"Viggo Mortensen,Lance Henriksen,Terry Chen","3,975",John Peterson lives with his partner Eric and ...
997,997,2012,I 2009,158 min,"\nAction, Adventure, Sci-Fi",5.8,49.0,Roland Emmerich,"John Cusack,Thandiwe Newton,Chiwetel Ejiofor","362,537",A frustrated writer struggles to keep his fami...
998,998,Star Wars: Episode VI - Return of the Jedi,1983,131 min,"\nAction, Adventure, Fantasy",8.3,58.0,Richard Marquand,"Mark Hamill,Harrison Ford,Carrie Fisher","982,354",After a daring mission to rescue Han Solo from...


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Unnamed: 0          1000 non-null   int64  
 1   Movie Name          1000 non-null   object 
 2   Year of Release     1000 non-null   object 
 3   Watch Time          957 non-null    object 
 4   Genre               1000 non-null   object 
 5   Movie Rating        939 non-null    float64
 6   Metascore of movie  905 non-null    float64
 7   Director            1000 non-null   object 
 8   Cast                1000 non-null   object 
 9   Votes               939 non-null    object 
 10  Description         1000 non-null   object 
dtypes: float64(2), int64(1), object(8)
memory usage: 86.1+ KB


In [4]:
data.rename(columns={'Unnamed: 0': 'movie_id'}, inplace=True)

In [12]:
data.rename(columns={'Movie Name': 'Title'}, inplace=True)

In [13]:
#columns_old=['Cast','Director','Genre','Title','Description']
columns=['Cast','Director','Genre','Title','Description']

In [14]:
data[columns].isnull().values.any()#no null values

np.False_